In [1]:
import pandas as pd
import datetime
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import re
import uuid

In [ ]:

url = "https://www.congreso.es/busqueda-de-publicaciones?p_p_id=publicaciones&p_p_lifecycle=0&p_p_state=normal&p_p_mode=view&_publicaciones_mode=mostrarTextoIntegro&_publicaciones_legislatura=XIV&_publicaciones_id_texto=(DSCD-14-CO-735.CODI.)"

page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')

x = soup.find(class_="textoIntegro")
x.get_text(separator=" ")

In [87]:
# BUENA LISTA DE URLS LEGISLATURA 14
full_url = []
n = 1

# AQUI añadir hoy - fecha de inicio legislatura en lugar de 800

for i in range(800):

    url = "https://www.congreso.es/busqueda-de-publicaciones?p_p_id=publicaciones&p_p_lifecycle=0&p_p_state=normal&p_p_mode=view&_publicaciones_mode=mostrarTextoIntegro&_publicaciones_legislatura=XIV&_publicaciones_id_texto=(DSCD-14-CO-{0}.CODI.)".format(n)
    full_url.append(url)
    n += 1

In [ ]:

url = "https://www.congreso.es/busqueda-de-publicaciones?p_p_id=publicaciones&p_p_lifecycle=0&p_p_state=normal&p_p_mode=view&_publicaciones_mode=mostrarTextoIntegro&_publicaciones_legislatura=XIV&_publicaciones_id_texto=(DSCD-14-CO-736.CODI.)"
page = requests.get(url)
soup = BeautifulSoup(page.content, 'lxml')

x = soup.find("div", attrs={"class":"textoIntegro publicaciones"})

li = []
for i in x:
    if i.string != None:
        li.append(i.string)

textitos = " ".join(li)
textitos

In [ ]:
df = pd.DataFrame(columns=["date","text", "type", "num", "url"])

for url in tqdm(full_url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'lxml')

    x = soup.find("div", attrs={"class":"textoIntegro publicaciones"})
    li = []

    for i in x:
        if i.string != None:
            li.append(i.string)

    texto_integro = " ".join(li)

    # TEXTO
    texto_integro = re.sub(r"Página\s\d*", " ",texto_integro)
    texto_integro = re.sub(r"(\,(?=\S))", ", ", texto_integro)
    texto_integro = re.sub(r"(\.(?=\S))", ". ", texto_integro)
    texto_integro = re.sub(r"(\)(?=\S))", ") ", texto_integro)
    texto_integro = re.sub(r"(\:(?=\S))", ": ", texto_integro)
    texto_integro = texto_integro.replace("\r", " ")
    texto_integro = texto_integro.replace("\n", " ")
    texto_integro = texto_integro.replace("\t", " ")
    texto_integro = re.sub(' +', ' ',texto_integro)

    meta = soup.find(class_="datos1").getText()
    date = meta[-10:]

    try:
        tipo = meta.split(",")[1].replace(" ","")
    except IndexError:
        break
    
    num = meta.split(",")[2].split(".")[1].replace(" ","")

    intervencion = []
    intervencion.append([date, texto_integro, tipo, num, url])

    df_ = pd.DataFrame(intervencion, columns=["date","text", "type", "num", "url"])
    df = pd.concat([df,df_], axis=0)

df['uuid'] = [uuid.uuid4() for x in range(df.shape[0])]
df.date = pd.to_datetime(df.date, format="%d/%m/%Y")
df["legislatura"] = 14
df.set_index("uuid", inplace=True)
df.to_csv("intervenciones_14_.csv")

In [ ]:
intervenciones_14 = df.copy()
intervenciones_14

In [ ]:
intervenciones_14_short = intervenciones_14[-1:]
intervenciones_14_short

In [335]:
full_df = pd.DataFrame(columns=["speaker", "discurso", "nºintervención","fecha", "num", "legislatura", "uuid", "url"])

for index,i in intervenciones_14.iterrows():
    l = []
    s = re.split(r"El señor\s(?=[A-ZÀ-Ÿ][A-ZÀ-ß])|La señora\s(?=[A-ZÀ-Ÿ][A-ZÀ-ß])", i["text"])
    
    for z,j in enumerate(s):
        x = j.split(sep=":", maxsplit=1)
        try:
            l.append([x[0],x[1],z,i.date,i.num,i.legislatura,index,i.url])
        except IndexError:
            l.append(["No Discurso",x,z,i.date,i.num,i.legislatura,index,i.url]) # BORRAR EN VERSION FINAL
            

    df_ = pd.DataFrame(l, columns=["speaker", "discurso", "nºintervención","fecha", "num", "legislatura", "uuid", "url"])
    full_df = pd.concat([full_df,df_], axis=0)
    
full_df.set_index("uuid", inplace=True)
full_df = full_df[full_df["nºintervención"] != 0]


In [ ]:
full_df